# Example

In [3]:
from generator import generate_nodes
import numpy as np

Let us generate a random network with k nodes and n data points.



In [2]:
n = 10_000_000  # size of data
k = 1024  # number of nodes
h = 20  # height of the tree

In [4]:
network, node_root, data, height = generate_nodes(n, k, h, split_deviation=0.0, random_gen=np.random.standard_normal,
                                                  graph_partition=True)

Sort the data so we can easily get n-th quantile.

In [5]:
data = np.sort(data)

We want to get q-th quantile.

In [8]:
q = 0.25
i = q * n

In [15]:
per = np.percentile(data, q * 100)
print("orig rank:", i)
print("orig numpy quantile:", per)
print("exact quantile:", data[int(i)])

orig rank: 2500000.0
orig numpy quantile: -0.6739484243114819
exact quantile: -0.6739481797451743


Desired error

In [16]:
eps = 0.0001
print(f"max rank error should be {eps * n}")

max rank error should be 1000.0


Create new task in the root node.

In [18]:
node_root.new_task(eps)

Now we can query the root node

In [21]:
rank = np.where(data == node_root.rank_query(i))[0][0]
print("result quantile: ", node_root.rank_query(i))
print("result rank:", np.where(data == node_root.rank_query(i))[0][0])
print("error:", np.abs(rank - i))
print("stat:", network.stat)

result quantile:  -0.6740154258383808
result rank: 2499803
error: 197.0
stat: {<MessageType.NEW_NODE: -1>: 1275, <MessageType.NEW_TASK: 0>: 102, <MessageType.RESULT: 1>: 664090, <MessageType.NEW_SUPER_NODE_TASK: -2>: 1944, <MessageType.NEW_SUPER_NODE_AGGREGATE: 3>: 2720927, <MessageType.SUPER_NODE_TASK_RESULT: 4>: 136378, <MessageType.SUPER_NODE_AGGREGATE_RESULT: 5>: 2720927}


As one can see, the result is very similar to what the exact and numpy quantiles.
The rank error is within the desired error.

Let us check the most famous quatiles for standard normal distribution.
Reference: https://www.researchgate.net/figure/True-Quantiles-and-Estimated-Quantiles-for-the-Standard-Normal-Distribution_tbl1_220504789

In [24]:
quantiles = [(0.01, -2.326), (0.025, -1.96), (0.05, -1.645), (0.1, -1.282), (0.5, 0.0), (0.9, 1.282), (0.95, 1.645),
             (0.975, 1.96), (0.99, 2.326)]

for q, val in quantiles:
    print(f"---\nQuantile {q}")
    print(f"Expected quantile value: {val}")
    print(f"Calculated quantile value: {node_root.rank_query(q * n)}")

---
Quantile 0.01
Expected quantile value: -2.326
Calculated quantile value: -2.324203086081388
---
Quantile 0.025
Expected quantile value: -1.96
Calculated quantile value: -1.9635812907712016
---
Quantile 0.05
Expected quantile value: -1.645
Calculated quantile value: -1.6452858117630766
---
Quantile 0.1
Expected quantile value: -1.282
Calculated quantile value: -1.2825414831287616
---
Quantile 0.5
Expected quantile value: 0.0
Calculated quantile value: 0.00038100629173741524
---
Quantile 0.9
Expected quantile value: 1.282
Calculated quantile value: 1.2807123994689054
---
Quantile 0.95
Expected quantile value: 1.645
Calculated quantile value: 1.644822066964553
---
Quantile 0.975
Expected quantile value: 1.96
Calculated quantile value: 1.9575637618396315
---
Quantile 0.99
Expected quantile value: 2.326
Calculated quantile value: 2.328399596276934


As you can see the network finds the quantile very closely!